#PREPARE GPU FOR the training model

#REFERENCES:

## Reference_1: Fine tune Bert https://www.youtube.com/watch?v=x66kkDnbzi4 by ChrisMcCormickAI


## Reference_2: applying SQUAD 1.0 dataset to BertForAnsweringQuestion already trained with SQUAD: https://www.youtube.com/watch?v=l8ZYCvgGu0o&list=WL&index=118&t=878s by ChrisMcCormickAI

## Reference_3: 'Question Answering with SQuAD 2.0' section from: https://huggingface.co/transformers/custom_datasets

## Reference_4: Basic knowledge about fine tuning, input formate and output format of BERT models : https://jalammar.github.io/a-visual-guide-to-using-bert-for-the-first-time/?fbclid=IwAR3uWlc8mUlrJ3QnYoYyQOfze3yDYkacgVyKSk24YjYE04Gs-7XiM3b9gTA 

#SET UP GPU FOR TRAINING MODEL:

**FIRSTLY, Setup GPU for training**
edit -> Notebook setting -> Hardware accelerator -> GPU

In [ ]:
import tensorflow as tf

# Get GPU device name:
device_name= tf.test.gpu_device_name()

# GPU device should have the following name:
if device_name == "/device:GPU:0":
  print("Found GPU at: " + device_name)
else:
  raise SystemError("GPU not found") # "GPU not found" a parameter to pass to the SystemError for printing out  

Found GPU at: /device:GPU:0


In [ ]:
import torch

# if there is a GPU device available..
if torch.cuda.is_available():

  # Tell TORCH to use this GPU:
  device= torch.device("cuda")

  print("There are %d GPU(s) available" % torch.cuda.device_count())

  print('we will use the GPU: ', torch.cuda.get_device_name(0))

# if not:
else:
  print('NO GPU available, using CPU instead')
  device = torch.device("cpu")

There are 1 GPU(s) available
we will use the GPU:  Tesla P4


#IMPORT DATASET

In [ ]:


# libraries for project:
import pandas as pd
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from collections import defaultdict
# necessary libraries for project:

dataset = pd.read_table('./data/final.tsv')  
dataset

,question,is_impossible,text,answer_start,context
0,When did Beyonce start becoming popular?,False,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,What areas did Beyonce compete in when she was...,False,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,When did Beyonce leave Destiny's Child and bec...,False,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,In what city and state did Beyonce grow up?,False,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,In which decade did Beyonce become famous?,False,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
...,...,...,...,...,...
9996,What famous World War II battle was the Canadi...,False,the Normandy Landings,166,Battles which are particularly notable to the ...
9997,What effort was the Canadian Military known fo...,False,the strategic bombing of German cities,288,Battles which are particularly notable to the ...
9998,What Battle in France was the Canadian Militar...,False,the Battle of Vimy Ridge,72,Battles which are particularly notable to the ...
9999,What country was the latest Canadian Military ...,False,Croatia,377,Battles which are particularly notable to the ...


##TRIM (Or SAMPLE) DOWN THE DATASET FOR TRAINING:

In [ ]:
# shuffling the dataset first beforing triming down:
# reference: https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset

,question,is_impossible,text,answer_start,context
0,When did she appear on the cover of GQ?,False,2013,236,"In September 2010, Beyoncé made her runway mod..."
1,How many people watched the season 14 finale?,False,8.03 million,621,The continuing decline influenced further chan...
2,How many people were in the earthquake emergen...,False,184,39,An earthquake emergency relief team of 184 peo...
3,"What were Fantasia Barrino, LaToya London and ...",False,the Three Divas,141,Much media attention on the season had been fo...
4,"The Buddha refused to pay respect to who, duri...",False,Vedas,575,A particular criticism of the Buddha was Vedic...
...,...,...,...,...,...
9996,Who was the 8th Karmapa Lama?,False,Mikyö Dorje,245,"The Zhengde Emperor (r. 1505–1521), who enjoye..."
9997,How did Washizawa refer to the city having the...,False,great nuisance,76,"In Japan, the Mayor of Nagano, Shoichi Washiza..."
9998,In what year did the population of New York re...,False,2010,738,"In the 1970s, job losses due to industrial res..."
9999,Principal photography began on what date?,True,5 July 2015,418,"After wrapping up in England, production trave..."


In [ ]:
# SAMPLE DOWN NUMBER OF DATASET FOR TRAINING AND EVALUATION:
dataset=dataset.iloc[:500,:]
dataset

,question,is_impossible,text,answer_start,context
0,When did Beyonce start becoming popular?,False,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,What areas did Beyonce compete in when she was...,False,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,When did Beyonce leave Destiny's Child and bec...,False,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,In what city and state did Beyonce grow up?,False,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,In which decade did Beyonce become famous?,False,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
...,...,...,...,...,...
495,What are Beyoncé's backup singers called?,False,The Mamas,216,"In 2006, Beyoncé introduced her all-female tou..."
496,When did The Mamas make their debut?,False,the 2006 BET Awards,343,"In 2006, Beyoncé introduced her all-female tou..."
497,What characteristics has Beyonce received accl...,False,stage presence and voice,36,Beyoncé has received praise for her stage pres...
498,Which former president of Def Jam called Beyon...,False,L.A. Reid,445,Beyoncé has received praise for her stage pres...


#EXTRACTING THE START AND END TOKENS OF AN ANSWER IN A CONTEXT

In [ ]:
# Small run:
# drop the question mark in the 'question' column:
def Drop_question_mark (quest):
  l= len(quest)
  return quest[:l-1]

dataset['question'] = dataset['question'].apply(Drop_question_mark)
dataset

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,question,is_impossible,text,answer_start,context
0,When did Beyonce start becoming popular,False,in the late 1990s,269,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
1,What areas did Beyonce compete in when she was...,False,singing and dancing,207,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
2,When did Beyonce leave Destiny's Child and bec...,False,2003,526,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
3,In what city and state did Beyonce grow up?,False,"Houston, Texas",166,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
4,In which decade did Beyonce become famous,False,late 1990s,276,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...
...,...,...,...,...,...
495,What are Beyoncé's backup singers called,False,The Mamas,216,"In 2006, Beyoncé introduced her all-female tou..."
496,When did The Mamas make their debut,False,the 2006 BET Awards,343,"In 2006, Beyoncé introduced her all-female tou..."
497,What characteristics has Beyonce received accl...,False,stage presence and voice,36,Beyoncé has received praise for her stage pres...
498,Which former president of Def Jam called Beyon...,False,L.A. Reid,445,Beyoncé has received praise for her stage pres...


##Import BertTokenizer for extracting the end and start tokens of an answer (in this case, the 'text') in a 'context'. [BertTokenizer also later used for tokenized and add special tokens [CLS], [SEP] for the input data of the Bert model]

In [ ]:
# INSTALL TRANSFORMER TO IMPORT BERT:
! pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast

In [ ]:
#Load the BERT tokenizer:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

##Extracting the necessary data from the 'dataset', such as 'context', 'text', 'question' as lists of data elements for easily hanlding this task. [Later these data lists also used as input for step TRANSFORMING the dataset into appropriate format input of Bert model]

In [ ]:
# list of all the 'context' from the dataset
context= dataset.context.values

# list of all the 'question' from the dataset
question = dataset.question.values

# list of all the 'answer' from the dataset
answers = dataset.text.values


##THE MAIN STEP : for extracting the end and start tokens of an 'answer' in a 'context'

In [ ]:
def extract_answer_start_end_tokens(answer, context):
  for i in range (0, len(context)-(len(answer)-2)+1):
    if context[i] == answer[1]: # find the first token of the answer in the context
      
      for j in range (1, len(answer)-1):
        if context[i+j-1] != answer[j]: # if the next tokens in the context are not in the answer
          break; # stop
      
      if j == len(answer)-2: # reach the end of the answer, in other words, the 'for-loop' of j reaches the end:
        # we have found the answer start and end indices in the context:
        start_token = i
        end_token= i+j-1
        return start_token, end_token
      # else: we move on to the next value in the context to keep searching for the answer start and end indices in the context.
  return 0, 0 # can not find the answer in the context

In [ ]:
# TEST:
# tokenize a 'context'
tokenized_context= tokenizer(context[1])

# extract only the input_ids from the tokenized result:
input_ids_tokenized_context = tokenized_context['input_ids'] 

# tokenize an 'answer'
tokenized_answer= tokenizer(answers[1])

# extract only the input_ids from the tokenized result:
input_ids_tokenized_answer = tokenized_answer['input_ids']

start, end = extract_answer_start_end_tokens(input_ids_tokenized_answer, input_ids_tokenized_context)
print(start)
print(end)
test_list=[]
for i in range (start, end+1):
  test_list.append(input_ids_tokenized_context[i])

tokens= tokenizer.convert_ids_to_tokens(test_list)

for token, id in zip(tokens, test_list):
  if id == tokenizer.sep_token_id:
    print(" ")
  print('{:12} {:>6}'.format(token,id))
  if id == tokenizer.sep_token_id:
    print(" ")

125
128
8              1022
.              1012
03             6021
million        2454


**EXTRACTING START AND END TOKENS OF AN 'ANSWER' IN EVERY 'CONTEXT' IN THE DATASET**

In [ ]:
start_labels = []
end_labels = []

In [ ]:
assert len(context) == len(answers) # must be TRUE

In [ ]:
for i in range (0, len(context)):
  # tokenize a 'context'
  tokenized_context= tokenizer(context[i])

  # extract only the input_ids from the tokenized result:
  input_ids_tokenized_context = tokenized_context['input_ids'] 

  # tokenize an 'answer'
  tokenized_answer= tokenizer(answers[i])

  # extract only the input_ids from the tokenized result:
  input_ids_tokenized_answer = tokenized_answer['input_ids']

  start, end = extract_answer_start_end_tokens(input_ids_tokenized_answer, input_ids_tokenized_context)

  start_labels.append(start)
  end_labels.append(end)

In [ ]:
# TEST:
assert len(start_labels) == len(end_labels)
assert len(start_labels) == len(context)

In [ ]:
# TEST:
# TEST:
# tokenize a 'context'
index= 4 # choose an examplary 'context', by choose a random value for the index in range (0, len(context))

tokenized_context= tokenizer(context[index])

# extract only the input_ids from the tokenized result:
input_ids_tokenized_context = tokenized_context['input_ids'] 

# tokenize an 'answer'
tokenized_answer= tokenizer(answers[index])

# extract only the input_ids from the tokenized result:
input_ids_tokenized_answer = tokenized_answer['input_ids']

tst_start = start_labels[index]
tst_end = end_labels[index]
print(f'{tst_start}\t{tst_end}')

test_list=[]
for i in range (tst_start, tst_end+1):
  test_list.append(input_ids_tokenized_context[i])

tokens= tokenizer.convert_ids_to_tokens(test_list)

for token, id in zip(tokens, test_list):
  if id == tokenizer.sep_token_id:
    print(" ")
  print('{:12} {:>6}'.format(token,id))
  if id == tokenizer.sep_token_id:
    print(" ")

69	70
late           2397
1990s          4134


#TRANSFORMING the dataset into appropriate format input of Bert model

##STEP 1: Tokenize the dataset and add special tokens [CLS], [SEP]. Then convert the tokenized the dataset into appropriate ids which are the indices of the lookup vocab table of the Bert model [Because the BertTokenizer is used for this task]

In [ ]:
# the maximum length of input sequence for bert-base-uncase is 512

# check the length of the input sequence:
# because input sequence = a 'context' + a'question' 
# the tokenized input is [CLS] + a 'context' + [SEP] + a'question'  +[SEP]
# => length of the tokenized input <= 512

# => Check the len(tokenized input), if it > 512, drop some tokens in the 'context' to make len = 512.

import random

def fit_max_length (input, max_len):

  # find the range of the 'context' tokens:
  for i in range (0, len(input)):

    if input[i] == 102: # encouter the first [SEP], which is the end of 'context
      range_context= i
      break
  
  # randomly drop some tokens in the 'context' to make the input len = 512:

  len_difference= len(input) - max_len
  
  # generate a list of random indices from 0 to range_context (not include (0 and value of range_context) ):
  # reference:
  
  for i in range (0, len_difference):
    d_index=random.randint( 1, range_context-1)
    del input[d_index]

    # the range_conext should be decreased by 1 due to the deleted tokens:
    range_context= range_context-1
  
  return input

In [ ]:
# Test: fit_max_length()
tokenized_sentences = tokenizer(context[10],question[10], add_special_tokens= True)
# BECAUSE input for the Bert model will be [CLS] + 'context' + [SEP] + 'question' + [SEP] (requirement_1),
# => tokenizer(context[10],question[10], add_special_tokens= True) takes care of the requirement_1.
# context[c_index], question[1_index] : c_index must be the same as q_question ('conext' must correspond to its own 'question')
# these indices can be a integer number in range (0, len (context))

ids= tokenized_sentences['input_ids']


max_len=200 # test with a random length

new_input_ids= fit_max_length(ids,max_len)

#len(new_input_ids), new_input_ids

tokens= tokenizer.convert_ids_to_tokens(new_input_ids)

for token, id in zip(tokens, new_input_ids):
  if id == tokenizer.sep_token_id:
    print(" ")
  print('{:12} {:>6}'.format(token,id))
  if id == tokenizer.sep_token_id:
    print(" ")

len(new_input_ids)

[CLS]           101
a              1037
brief          4766
shoot          5607
at             2012
london         2414
'              1005
s              1055
city           2103
hall           2534
was            2001
filmed         6361
on             2006
18             2324
april          2258
2015           2325
,              1010
while          2096
was            2001
on             2006
.              1012
on             2006
may            2089
2015           2325
took           2165
place          2173
on             2006
thames        11076
in             1999
london         2414
.              1012
scenes         5019
involving      5994
and            1998
##yd          25688
##oux         28700
on             2006
a              1037
##boat        11975
as             2004
well           2092
as             2004
a              1037
low            2659
flying         3909
helicopter     7739
near           2379
bridge         2958
were           2020
shot           2915


200

In [ ]:
# Each entry at ith index of the input_ids is 'ids' like in the 'TEST' cell right above.
# Maximum sequence length for this model (512)
# => Each entry of the input_ids must be 512.

input_ids=[]
max_len= 512
for cntx, quest in zip(context, question):

  tokenized_sentences = tokenizer(cntx,quest, add_special_tokens= True)
  ids= tokenized_sentences['input_ids']

  if len(ids) > 512: # if the sequence length > 512
    ids= fit_max_length(ids, max_len) # decrease it to 512

  input_ids.append(ids)

In [ ]:
# TEST:
test_1=input_ids[4]

tokens= tokenizer.convert_ids_to_tokens(test_1)

for token, id in zip(tokens, test_1):
  if id == tokenizer.sep_token_id:
    print(" ")
  print('{:12} {:>6}'.format(token,id))
  if id == tokenizer.sep_token_id:
    print(" ")
print(len(test_1))

[CLS]           101
beyonce       20773
gi            21025
##selle       19358
knowles       22815
-              1011
carter         5708
(              1006
/              1013
bi            12170
##ː           23432
##ˈ           29715
##j            3501
##ɒ           29678
##nse         12325
##ɪ           29685
/              1013
bee           10506
-              1011
yo            10930
##n            2078
-              1011
say            2360
)              1007
(              1006
born           2141
september      2244
4              1018
,              1010
1981           3261
)              1007
is             2003
an             2019
american       2137
singer         3220
,              1010
songwriter     6009
,              1010
record         2501
producer       3135
and            1998
actress        3883
.              1012
born           2141
and            1998
raised         2992
in             1999
houston        5395
,              1010
texas          3146


In [ ]:
# CHECK:
input_ids[10]
for i in range (0,len(input_ids)):
  if len(input_ids[i]) > 512:
    print("NOT OK")
    break

##PADDING: to make all the input sequences (in this case, an entry of the 'input_ids' list) the same length [because Bert model requires such a thing]

In [ ]:
# CHECK The maximum length of each sequence in input_ids:
max_len_input_ids=0
for i in range (0, len(input_ids)):
  if len(input_ids[i])> max_len_input_ids:
    max_len_input_ids= len(input_ids[i])

max_len_input_ids 

450

In [ ]:
# PADDING: FOR the input_ids
# [PAD] in Bert has value of 0

from keras.preprocessing.sequence import pad_sequences

# set the Max_len:
Max_len= 450 # set as the same value with the 'max_len_input_ids'

# because '[PAD]' in Bert vocab look-up take has id (or index) =0 
# => we can padd 0 values at the end of each entries of 'input_ids'
pad_input_ids= pad_sequences(input_ids, maxlen=Max_len, dtype='long', value=0, truncating='post', padding='post')

# check whether the padding and truncating after padding work as we expect:
assert len(pad_input_ids[10]) == Max_len
print(len(input_ids[0])) # original length of the input_ids[0]


173


In [ ]:
# TEST:
test_1=pad_input_ids[5]

tokens= tokenizer.convert_ids_to_tokens(test_1)

for token, id in zip(tokens, test_1):
  if id == tokenizer.sep_token_id:
    print(" ")
  print('{:12} {:>6}'.format(token,id))
  if id == tokenizer.sep_token_id:
    print(" ")
print(len(test_1))

[CLS]           101
beyonce       20773
gi            21025
##selle       19358
knowles       22815
-              1011
carter         5708
(              1006
/              1013
bi            12170
##ː           23432
##ˈ           29715
##j            3501
##ɒ           29678
##nse         12325
##ɪ           29685
/              1013
bee           10506
-              1011
yo            10930
##n            2078
-              1011
say            2360
)              1007
(              1006
born           2141
september      2244
4              1018
,              1010
1981           3261
)              1007
is             2003
an             2019
american       2137
singer         3220
,              1010
songwriter     6009
,              1010
record         2501
producer       3135
and            1998
actress        3883
.              1012
born           2141
and            1998
raised         2992
in             1999
houston        5395
,              1010
texas          3146


#BESIDES, the input sequences, the SEGMENT MASK is also required as input for the Bert model

as a sequence input in the 'input_ids' has format:
[CLS] + 'context' + [SEP] + 'question' + [SEP] + [PAD]s.

Then we would assign a sequence of 1s (1, 1, 1, ...) for the first part [CLS] + 'context' + [SEP]; and assign a sequence of 0s (0, 0, 0, ...) for the second part 'question' + [SEP] + [PAD]s.

In [ ]:
segment_masks=[] # consider [PAD]s belonging to the second sequence
for i in range (0, len(pad_input_ids)):

  convert_to_list= pad_input_ids[i].tolist()
  sep_index= convert_to_list.index(tokenizer.sep_token_id)

  # number of the 'context' (='answer') tokens includes the [SEP] also
  num_seg_a= sep_index+1

  # the remainder is the 'question':
  num_seg_b= len(convert_to_list) - num_seg_a

  # construct list of 0s and 1s:
  segment_ids= [1]*num_seg_a + [0]*num_seg_b # a segment mask for [CLS]+ a 'context' +[SEP] + 'text' + [SEP]

  # add the segment_ids to the list of segment masks:
  segment_masks.append(segment_ids)

In [ ]:
# TEST:
number_of_ones=0
position= 99 # take a segment mask in the list segment_maks for testing
for i in range (0, len(segment_masks[position])):
  if segment_masks[position][i] == 1:
    number_of_ones= number_of_ones+1

number_of_zeros = len(segment_masks[position]) - number_of_ones

# the real number of ones in the segment masks:
convert_to_list= pad_input_ids[position].tolist()
sep_index= convert_to_list.index(tokenizer.sep_token_id)
real_number_of_ones= sep_index+1
real_number_of_zeros= len(convert_to_list) - real_number_of_ones
#
assert number_of_ones== real_number_of_ones
assert number_of_zeros== real_number_of_zeros

In [ ]:
# TEST:

for i in range (0, 15): # all segment_masks must have the same length
  print(len(segment_masks[i]))

450
450
450
450
450
450
450
450
450
450
450
450
450
450
450


#ATTTENTION MASK is also required together with the segment mask

as a sequence input in the 'input_ids' has format:
[CLS] + 'context' + [SEP] + 'question' + [SEP] + [PAD]s.

Then we would assign a sequence of 1s (1, 1, 1, ...) for the first part [CLS] + 'context' + [SEP] + 'question' + [SEP]; and assign a sequence of 0s (0, 0, 0, ...) for the second part [PAD]s.

In [ ]:
# Create attention mask:
attention_masks= []

for pad_sequence in pad_input_ids:

  # because [PAD] has id = 0 => we could use this condition to apply the attension mask:
  attention_mask=[int(token_id >0) for token_id in pad_sequence]

  # aggregate each mask of each padded sequence into a list attention_masks
  # with this method, we could preserve the corresponding order between a padded sequence and its mask:
  attention_masks.append(attention_mask)


In [ ]:
# CHECK attention_masks:
# check the mask for first sentence:

# the first encoded sentence:
#print(pad_input_ids[0])

# the mask of the first encoded sentence:
#print(attention_masks[0]) # there should be 19 values of '1' at the beginning.

# count '1' values in the first attention mask, the result should be 174
c=0
for i in range (0, len(attention_masks[2])):
  if attention_masks[2][i]==1:
    c=c+1
print(c)

73


In [ ]:
assert len(attention_masks[10]) == 445

#SPLIT the dataset, the segment masks, the attention maks, the start labels, the end labels into the train set and evaluation set

In [ ]:
from sklearn.model_selection import train_test_split

# split for the conformed input dataset of BERT (input_ids) and the labels list
train_inputs, evl_inputs, train_start_labels, evl_start_labels= train_test_split(pad_input_ids, start_labels, random_state=2018, test_size=0.1)

# do the same for segment masks of conformed input dataset:
train_segment_masks, evl_segment_masks, train_end_labels, evl_end_labels= train_test_split(segment_masks, end_labels, random_state=2018, test_size=0.1)
# _,_ is used because masks need no labels.

# do the same for segment masks of conformed input dataset:
train_attention_masks, evl_attention_masks,_,_= train_test_split(attention_masks, start_labels, random_state=2018, test_size=0.1)
# _,_ is used because masks need no labels.

In [ ]:
#train_inputs.shape, train_labels.shape
print(train_inputs.shape)

(450, 450)


#CONVERT the train set and the evaluation set into 'torch.tensor' type becuase Bert model requires 'torch.tensor' type as its valid input type.

In [ ]:
# for the input data:
train_inputs= torch.tensor([train_inputs])
evl_inputs= torch.tensor([evl_inputs])

# for the start_labels:
train_start_labels= torch.tensor(train_start_labels)
evl_start_labels= torch.tensor(evl_start_labels)

# for the end_labels:
train_end_labels= torch.tensor(train_end_labels)
evl_end_labels= torch.tensor(evl_end_labels)

# for segment masks:
train_segment_masks= torch.tensor([train_segment_masks])
evl_segment_masks= torch.tensor([evl_segment_masks])

# for attention masks:
train_attention_masks=torch.tensor([train_attention_masks])
evl_attention_masks=torch.tensor([evl_attention_masks])

In [ ]:
# check the shape:
train_inputs.shape, train_start_labels.shape, train_end_labels.shape, train_segment_masks.shape, train_attention_masks.shape

(torch.Size([1, 450, 450]),
 torch.Size([450]),
 torch.Size([450]),
 torch.Size([1, 450, 450]),
 torch.Size([1, 450, 450]))

#Generate the train set and evaluation set in batches:

**FUNCTION FOR GENERATING BATCHES with batch size chosen by user**

In [ ]:
# LEARN FROM PREVIOUS LECTURES AND LABS in the class:
class BatchedIterator:
    def __init__(self, *tensors, batch_size,**kwarg):
        # all tensors must have the same first dimension
        assert len(set(len(tensor) for tensor in tensors)) == 1
        #print(type(tensors))
        #print(tensors[1])
        self.tensors = tensors
        self.batch_size = batch_size

        for keyword, value in kwarg.items():
            if keyword == "shuffle":
                self.shuffle=value
    
    def iterate_once(self):
        num_data = len(self.tensors[0][0]) # the length of the data

        if self.shuffle== False:
          for start in range(0, num_data, self.batch_size):
              end = start + self.batch_size
              yield tuple(tensor[0][start:end] for tensor in self.tensors) 
              #must be tensor[0], to access to real data
              # cause tensor size [1,..]; 1: is unecessary dimension
              # => must exclude it by using tensor[0]  
        else:
          all_batches=[] # to gather all the batches formed form the dataset
          for start in range(0, num_data, self.batch_size):
              end = start + self.batch_size
              all_batches.append(tuple(tensor[0][start:end] for tensor in self.tensors))
          
          # shuffle the batches: 
          # reference: https://note.nkmk.me/en/python-random-shuffle/#:~:text=To%20randomly%20shuffle%20elements%20of,Python%2C%20use%20the%20random%20module.&text=random%20provides%20shuffle()%20that,used%20for%20strings%20and%20tuples.
          shuf_batches = random.sample(all_batches, len(all_batches))

          # yield a batch in the list at a time:
          for i in range (0,len(shuf_batches)):
            yield shuf_batches[i]

In [ ]:
# extract for test sample:
test_train_inputs= train_inputs[:,:10,:]

test_train_start_labels= train_start_labels[:10]
test_train_end_labels= train_end_labels[:10]

test_train_segment_masks = train_segment_masks[:,:10,:]
test_train_attention_masks = train_attention_masks[:,:10,:]

test_train_inputs.shape, test_train_start_labels.shape, test_train_end_labels.shape, test_train_segment_masks.shape, test_train_attention_masks.shape,

(torch.Size([1, 10, 444]),
 torch.Size([10]),
 torch.Size([10]),
 torch.Size([1, 10, 444]),
 torch.Size([1, 10, 444]))

In [ ]:
# the torch.size must be torch.size([1,...])
# but the test_train_labels has the torch.size([10])
# must make it into torch.size([1,10])
good_test_train_start_labels=torch.unsqueeze(test_train_start_labels, 0)
good_test_train_end_labels=torch.unsqueeze(test_train_end_labels, 0)

#test:
good_test_train_start_labels.shape, good_test_train_end_labels.shape

(torch.Size([1, 10]), torch.Size([1, 10]))

In [ ]:
# Test out the BatchedIterator:

batch_size = 5
train_iter = BatchedIterator(test_train_inputs, good_test_train_start_labels,good_test_train_end_labels, test_train_segment_masks,test_train_attention_masks, batch_size=batch_size)

for train_batch, start_batch, end_batch, segment_mask_batch, attention_batch in train_iter.iterate_once():
  print(f'train_batch.type= {train_batch.shape}\tstart_batch.type={start_batch.shape}\tend_batch={end_batch.shape}\tsegment_mask_batch={segment_mask_batch.shape}\tattention_mask_batch={attention_batch.shape}')
  print(train_batch)

train_batch.type= torch.Size([5, 449])	start_batch.type=torch.Size([5])	end_batch=torch.Size([5])	segment_mask_batch=torch.Size([5, 449])	attention_mask_batch=torch.Size([5, 449])
tensor([[  101, 25479,  2001,  ...,     0,     0,     0],
        [  101,  2014,  2034,  ...,     0,     0,     0],
        [  101, 26322,  2015,  ...,     0,     0,     0],
        [  101, 20773,  1005,  ...,     0,     0,     0],
        [  101,  3225,  1999,  ...,     0,     0,     0]])
train_batch.type= torch.Size([5, 449])	start_batch.type=torch.Size([5])	end_batch=torch.Size([5])	segment_mask_batch=torch.Size([5, 449])	attention_mask_batch=torch.Size([5, 449])
tensor([[  101,  6207,  3107,  ...,     0,     0,     0],
        [  101, 26880,  1998,  ...,     0,     0,     0],
        [  101,  1999,  2244,  ...,     0,     0,     0],
        [  101,  1999, 17403,  ...,     0,     0,     0],
        [  101,  5943, 16966,  ...,     0,     0,     0]])


#DEFINE BERT MODEL FOR TRAINING

**Bert model is too large, i would use DistlledBert model which is 40 percent smaller than Bert model but still preserves 97 percent performace of the Bert model**

In [ ]:
# https://huggingface.co/transformers/model_doc/bert.html#bertforquestionanswering
from transformers import DistilBertForQuestionAnswering

# reference for config a pretrained model : https://towardsdatascience.com/hugging-face-transformers-fine-tuning-distilbert-for-binary-classification-tasks-490f1d192379
from transformers import DistilBertConfig

In [ ]:
class ForQuestionAnsweringClassifier(nn.Module):
    def __init__(self,model_out_sequence_length, output_size, freeze_bert = True):
        super(ForQuestionAnsweringClassifier, self).__init__()
        # Configure DistilBERT's initialization
        self.bert_config = DistilBertConfig(n_layers=1, n_heads=2,qa_dropout=0.2, dim = 312)
                          
        self.Distil_Bert_model= DistilBertForQuestionAnswering(config=self.bert_config)

        # Make DistilBERT layers untrainable
        for p in self.Distil_Bert_model.parameters():
          p.requires_grad = False
          p.trainable = False
                
        #Classification layer
        self.start_cls_layer = nn.Linear(model_out_sequence_length, output_size)
        self.end_cls_layer = nn.Linear(model_out_sequence_length, output_size)

    def forward(self, data, attn_masks, start_label, end_label):
        bertOut= self.Distil_Bert_model(data, # the tokens representing our input
                attention_mask=attn_masks, start_positions= start_label, end_positions= end_label ) # the segment ids to differentiate the question and the answer
        start_labels= self.start_cls_layer(bertOut.start_logits)
        end_labels=self.start_cls_layer(bertOut.end_logits)
        return start_labels, end_labels

In [ ]:
model = ForQuestionAnsweringClassifier(450,450) #model_out_sequence_length = output_size

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# Tell Pytorch to run this model on the GPU:
#for p in model.parameters():
#      p.requires_grad = False
model.to(device)

ForQuestionAnsweringClassifier(
  (Distil_Bert_model): DistilBertForQuestionAnswering(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 312, padding_idx=0)
        (position_embeddings): Embedding(512, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=312, out_features=312, bias=True)
              (k_lin): Linear(in_features=312, out_features=312, bias=True)
              (v_lin): Linear(in_features=312, out_features=312, bias=True)
              (out_lin): Linear(in_features=312, out_features=312, bias=True)
            )
            (sa_layer_norm): LayerNorm((312,), eps=1e-12, elementwise_affine=Tr

#TRAINING AND EVALUATION:

In [ ]:
# Convert the data into tensor so that the model can run on the data
# extract for test sample:
train_inputs
evl_inputs

train_inputs= train_inputs.to(device)
evl_inputs = evl_inputs.to(device)

train_start_labels
train_end_labels

train_start_labels= train_start_labels.to(device)
train_end_labels= train_end_labels.to(device) 


evl_start_labels
evl_end_labels

evl_start_labels= evl_start_labels.to(device)
evl_end_labels= evl_end_labels.to(device)

train_segment_masks
evl_segment_masks


train_segment_masks= train_segment_masks.to(device) 
evl_segment_masks= evl_segment_masks.to(device) 



train_attention_masks
evl_attention_masks


train_attention_masks = train_attention_masks.to(device) 
evl_attention_masks= evl_attention_masks.to(device) 

train_inputs.shape, train_start_labels.shape, train_end_labels.shape, train_segment_masks.shape, train_attention_masks.shape

(torch.Size([1, 450, 450]),
 torch.Size([450]),
 torch.Size([450]),
 torch.Size([1, 450, 450]),
 torch.Size([1, 450, 450]))

In [ ]:
# the torch.size must be torch.size([1,...])
# but the test_train_labels has the torch.size([10])
# must make it into torch.size([1,10])
good_train_start_labels=torch.unsqueeze(train_start_labels, 0)
good_train_end_labels=torch.unsqueeze(train_end_labels, 0)

good_evl_start_labels=torch.unsqueeze(evl_start_labels, 0)
good_evl_end_labels=torch.unsqueeze(evl_end_labels, 0)

#test:
good_train_start_labels.shape, good_train_end_labels.shape

(torch.Size([1, 450]), torch.Size([1, 450]))

In [ ]:
batch_size = 100
train_iter = BatchedIterator(train_inputs, good_train_start_labels, good_train_end_labels, train_segment_masks, train_attention_masks, batch_size=batch_size, shuffle=True)

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion = criterion.cuda()
optimizer = optim.Adam(model.parameters())

In [ ]:
num_epochs = 15

In [ ]:
for epoch in range(num_epochs):
    model.train()
    # Training on train data
  
    for train_batch, start_batch, end_batch, segment_mask_batch, attention_batch in train_iter.iterate_once():
        
        #train_batch = train_batch.to(device)
        #attention_batch = attention_batch.to(device)
        #start_batch = start_batch.to(device)
        #end_batch = end_batch.to(device)
        # run the model on the inputs:
        start_predicts, end_predicts = model(train_batch, # the tokens representing our input
                attention_batch, start_batch, end_batch ) # the segment ids to differentiate the question and the answer
        

        # FOR TASK 1:
        
        #y_out = model(X_batch)
        #print(y_out)
        # FOR TASK 1:
        
        # To understand the ouput of the model:
        # y_out.shape = [batch_size, output_size] # output_size = number of labels
        #print(f'y_out.shape= {y_out.shape} \ny_batch.shape= {y_batch.shape} ')

        #print(f'y_out.shape= {y_out.shape} \ny_batch.shape= {y_batch.shape} ')
        #train_loss= outputs.loss
        print(f'start_predict.shape={start_predicts.shape}\tstart_batch={start_batch.shape}')
        start_loss= criterion(start_predicts, start_batch)
        end_loss= criterion(end_predicts, end_batch)
        #print("loss:")
        #print(loss)
        optimizer.zero_grad()
        start_loss.backward()
        end_loss.backward()
        optimizer.step()
        
    model.eval()  # or model.train(False)
    
    #Move the training data for evaluation
    #train_inputs[0] = train_inputs[0].to(device)
    #train_attention_masks[0] = train_attention_masks[0].to(device)
    #train_start_labels = train_start_labels.to(device)
    #train_end_labels = train_end_labels.to(device)
        
    # evaluation on train data:
    start_predicts, end_predicts = model(train_inputs[0], # the tokens representing our input
                train_attention_masks[0], train_start_labels , train_end_labels) # the segment ids to differentiate the question and the answer

    start_labels= start_predicts.argmax(axis=1)
    end_labels=end_predicts.argmax(axis=1)

    #train_loss= outputs.loss
    start_train_loss = criterion(start_predicts, train_start_labels).item()
    end_train_loss = criterion(end_predicts, end_labels).item()
    train_loss= (start_train_loss + end_train_loss)/2

    train_start_accuracy = (torch.eq(start_labels, train_start_labels).sum() / float(len(train_start_labels))).item()
    train_end_accuracy = (torch.eq(end_labels, train_end_labels).sum() / float(len(train_end_labels))).item()
    train_accuracy = (train_start_accuracy + train_end_accuracy) /2
    

    
    # evaluation on eval data:
    #Move the training data for evaluation
    #evl_inputs[0] = evl_inputs[0].to(device)
    #evl_attention_masks[0] = evl_attention_masks[0].to(device)
    #evl_start_labels = evl_start_labels.to(device)
    #evl_end_labels = evl_end_labels.to(device)
    

    evl_start_predicts, evl_end_predicts = model(evl_inputs[0], # the tokens representing our input
                evl_attention_masks[0], evl_start_labels , evl_end_labels) # the segment ids to differentiate the question and the answer

    start_labels= evl_start_predicts.argmax(axis=1)
    end_labels=evl_end_predicts.argmax(axis=1)

    start_dev_loss = criterion(evl_start_predicts, evl_start_labels).item()
    end_dev_loss = criterion(evl_end_predicts, evl_end_labels).item()
    dev_loss= (start_train_loss + end_train_loss)/2
    
    evl_start_accuracy = (torch.eq(start_labels, evl_start_labels).sum() / float(len(evl_start_labels))).item()
    evl_end_accuracy = (torch.eq(end_labels, evl_end_labels).sum() / float(len(evl_end_labels))).item()
    evl_accuracy = (evl_start_accuracy + evl_end_accuracy) /2
    

    print(f"Epoch: {epoch} -- train loss: {train_loss} - train acc: {train_accuracy*100} - "
          f"dev loss: {dev_loss} - dev acc: {evl_accuracy*100}")

start_predict.shape=torch.Size([100, 450])	start_batch=torch.Size([100])
start_predict.shape=torch.Size([100, 450])	start_batch=torch.Size([100])
start_predict.shape=torch.Size([100, 450])	start_batch=torch.Size([100])
start_predict.shape=torch.Size([50, 450])	start_batch=torch.Size([50])
start_predict.shape=torch.Size([100, 450])	start_batch=torch.Size([100])


RuntimeError: ignored

#SAVE & LOAD trained model:

##TO SAVE

In [ ]:
torch.save(model.state_dict(), './model/Trained_model.tsv')

##TO LOAD:

In [ ]:
device = torch.device("cuda")
model = ForQuestionAnsweringClassifier(449,449)
model.load_state_dict(torch.load('./model/Trained_model.tsv'))
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'distilbert.transformer.layer.1.attention.q_lin.weight', 'distilbert.transformer.layer.1.attention.q_lin.bias', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.1.attention.k_lin.bias', 'distilbert.transformer.layer.1.attention.v_lin.weight', 'distilbert.transformer.layer.1.attention.v_lin.bias', 'distilbert.transformer.layer.1.attention.out_lin.weight', 'distilbert.transformer.layer.1.attention.out_lin.bias', 'distilbert.transformer.layer.1.sa_layer_norm.weight', 'distilbert.transformer.layer.1.sa_layer_norm.bias', 'distilbert.transformer.layer.1.ffn.lin1.weight', 'distilbert.transformer.layer.1.ffn.lin1.bias', 'distilbert.transformer.layer.1.ffn.lin2.weight', 'distilbert

ForQuestionAnsweringClassifier(
  (Distil_Bert_model): DistilBertForQuestionAnswering(
    (distilbert): DistilBertModel(
      (embeddings): Embeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (transformer): Transformer(
        (layer): ModuleList(
          (0): TransformerBlock(
            (attention): MultiHeadSelfAttention(
              (dropout): Dropout(p=0.1, inplace=False)
              (q_lin): Linear(in_features=768, out_features=768, bias=True)
              (k_lin): Linear(in_features=768, out_features=768, bias=True)
              (v_lin): Linear(in_features=768, out_features=768, bias=True)
              (out_lin): Linear(in_features=768, out_features=768, bias=True)
            )
            (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

**To Run on the loaded model, just go back to 'TRAINING AND EVALUATION' section.**

In [ ]:
import gc
del model
gc.collect()

NameError: ignored